In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import bs4
import datetime as dt

In [4]:
url = 'https://www.bankofcanada.ca/press/press-releases/?location%5B%5D=1509&mtf_search=basis%20point'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
# get the id named multitaxonomytaglist-cfct-module-effbb0fd500f83b8ae3d9bc9fa5cbee4_results
results = soup.find(id='multitaxonomytaglist-cfct-module-effbb0fd500f83b8ae3d9bc9fa5cbee4_results')

In [21]:
import time

In [63]:
can_statements  = {}
for i in range(1, 10):
    if i == 1:
        url = 'https://www.bankofcanada.ca/press/press-releases/?location%5B%5D=1509&mtf_search=basis%20point'

    else:
        url = f'https://www.bankofcanada.ca/press/press-releases/?location%5B%5D=1509&mtf_search=basis%20point&mt_page={i}'
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find(id='multitaxonomytaglist-cfct-module-effbb0fd500f83b8ae3d9bc9fa5cbee4_results')
    for i in results:
        if type(i) is not bs4.element.NavigableString:
            try:
                title = (i.find(class_='media-heading').text).strip()
                can_statements[title] = {}

                date = i.find(class_='bocss-margin-left-medium media-date pull-right').text
                date = str(dt.datetime.strptime(date, '%B %d, %Y').date())
                can_statements[title]['date'] = date

                link = i.find('a')['href'].strip()
                can_statements[title]['link'] = i.find('a')['href'].strip()

                # parsing the link itself
                time.sleep(0.5)
                can_statements[title]['text'] = ''
                statement = requests.get(link)
                soup_ = BeautifulSoup(statement.content, 'html.parser')
                text = soup_.find(class_='post-content')
                for i in text:
                    if type(i) is not bs4.element.NavigableString:
                        if i.name == 'p':
                            can_statements[title]['text'] += '\n' + i.text
                
                can_statements[title]['text'] = can_statements[title]['text'].strip()
            except:
                pass
    
    
    time.sleep(0.5)


In [77]:
import json


In [80]:
with open('data/dataset/can_statements.json', 'w') as fp:
    json.dump(can_statements, fp,  indent=4)